In [1]:
from fake_useragent import UserAgent
from os.path import join,isfile
from os import listdir
import dask
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from collections import Counter
import json
import re
import time
import bookquery
import sys
import os
import datetime
import math

ua = UserAgent()#Прокси
curr_directory = os.getcwd()
new_path = join(curr_directory,"Information")
try:# Создаем базовую директорию для файлов
    os.mkdir(new_path)
except OSError:
    print ("Директория %s уже создана" % new_path)
    
def execute_query(qquery, entity_id):
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def counter_for_files(path):
    
    """Подсчет файлов в директории"""
    
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

Директория C:\Users\Алексей Кушнер\Sberbank\Geography\Big\Information уже создана


In [2]:
path = []
length = 0
path.append((join(os.getcwd() , r'pool\2\data\data_with_popularity'),'popularity'))
path.append((join(os.getcwd() , r'pool\2\analyzing'),'entity_id'))
path.append((join(os.getcwd() , r'pool\2\analyzing\aggregated') , 'prop_count'))
def concat_sets(path , stat):
    
    """Группировка файлов"""
    
    global length
    df  =[]
    for i in counter_for_files(path):
        part_df = pd.read_csv(i, encoding='utf8',sep = ';')
        df.append(part_df)
    df = pd.concat(df)
    if stat =='popularity':
        length = len(df)
    name = counter_for_files(path)[0].split(f'\\')[-1].split('.')[0].split('_')[:-1]
    name = '_'.join(name)
    if stat == 'prop_count':
        df = df.groupby('prop_name').sum().sort_values(by = stat,ascending = False)
        df['prop_counter'] = ( df['prop_count'] / length)*100
        df.to_csv(f'{path}\\{name}.csv' , encoding='utf-8-sig',sep = ';')
    else:
        df.sort_values(by = stat,ascending = False).to_csv(f'{path}\\{name}.csv' ,
                                                           encoding='utf-8-sig',sep = ';' , index = False)
    
for i in path:
    concat_sets(i[0] , i[1])    

In [3]:
#Блок формирования файла, агрегировнного по предикатам 

new_path = join(os.getcwd(),r'pool\2\data\data_with_popularity')
new_path_stats= join(os.getcwd(),r'pool\2\analyzing')
new_path_aggregated = join(os.getcwd(),r'pool\2\analyzing\aggregated_with_popularity')
def finding_popular(most_popular):
    
    """Обработка 5% наиболее популярных сущностей"""
    
    if len(most_popular) * 0.05 < 10:
        return list(most_popular.iloc[:10].entity)
    elif len(most_popular) * 0.05 >= 100:
        return list(most_popular.iloc[:100].entity)
    else:
        return list(most_popular.iloc[:int(len(most_popular) * 0.05)].entity)
    
stats_df = pd.read_csv(counter_for_files(new_path_stats)[0],encoding='utf8',sep=';')    
most_popular = pd.read_csv(counter_for_files(new_path)[0],encoding='utf8',sep=';')
most_popular = finding_popular(most_popular)
def filter_func(x):
    return x['entity_id'].max() in most_popular
        #Топ 5% популярных сущностей
name = counter_for_files(new_path)[0].split(f'\\')[-1].split('.')[0].split('_')[:-2]
name = '_'.join(name)
popular_stats = stats_df.groupby('entity_id').filter(filter_func).groupby('prop_name').sum().sort_values(by = ['prop_count','prop_count_value'],ascending = False)
popular_stats['prop_counter'] = popular_stats['prop_count']
popular_stats.to_csv(join(new_path_aggregated,  f'{name}_aggregated_with_popularity.csv'), encoding='utf-8-sig',sep = ';')